# L1C Formatting Eploration

In [2]:
import pandas as pd
import h5py
# from pyhdf.SD import SD, SDC
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
os.chdir('/project/sdscap-shakeri/nasa/UVA_NASA_2021')
import icare
import pickle
import copy
os.getcwd()

'/gpfs/gpfs0/project/sdscap-shakeri/nasa/UVA_NASA_2021'

In [3]:
# os.chdir('C:\\Users\\whetz\\Documents\\UVA MSDS\\NASA\\hdf_files')
session = icare.ICARESession('/project/sdscap-shakeri/nasa/HDF')

ICARE Username: jpeele
ICARE Password: ···············


In [4]:
path='PARASOL/L1_B-HDF.v1.00/2008/2008_06_01'
file_list = list(session.listdir(path))
file_path = path + '/' + file_list[0]
session.get_file(file_path)

'/project/sdscap-shakeri/nasa/HDF/PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5'

In [7]:
os.chdir('/project/sdscap-shakeri/nasa/HDF')

In [8]:
f = h5py.File("PARASOL/L1_B-HDF.v1.00/2008/2008_06_01/POLDER3_L1B-BG1-080146M_2008-06-01T00-08-19_V1-00.h5", "r")

In [9]:
list(f['Data_Directional_Fields']['thetas'].attrs)

['DIMENSION_LABELS',
 'HDF_Calibration_Equation',
 'Num_Fill',
 'Num_Valid',
 'Num_missing_value',
 '_FillValue',
 'actual_range',
 'add_offset',
 'add_offset_err',
 'long_name',
 'missing_value',
 'scale_factor',
 'scale_factor_err',
 'units']

In [10]:
print("Solar Zenith:", f['Data_Directional_Fields']['thetas'].attrs['long_name'])
print("Sensor Zenith:", f['Data_Directional_Fields']['thetav'].attrs['long_name'])
print("Relative Azimuth:", f['Data_Directional_Fields']['phi'].attrs['long_name'])

Solar Zenith: b'solar zenith angle of the pixel center '
Sensor Zenith: b'view zenith angle of the pixel center  for filter 670P2'
Relative Azimuth: b'relative azimuth angle of the pixel center  for filter 670P2'


if solar zenith is at 90 degrees - horizon

not interested in angles that are > 60 

only want < 60 degrees

Maybe just take a subset of pixels (just the middle 100 longitudes, all the latitudes)

# Create example file from h5

In [17]:
measurement_dict = {}
for cat in ['I_np','I_p','Q','U']:
    print(cat)
    
    if cat == 'I_np':
        tag = cat.replace('_np','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field)])
    else:
        tag = cat.replace('_p','')
        fields = ([field for field in list(f['Data_Directional_Fields'].keys()) if (tag in field) and ('NP' not in field)])

    print("tag:", tag)
    fields.sort()
    print(fields)
    arrays = []
    scales = []
    long_names = []
    fills = []
    units = []
    for field in fields:
            
        print(field, end=", ")
        
        scales.append(f['Data_Directional_Fields'][field].attrs['scale_factor'])
        long_names.append(f['Data_Directional_Fields'][field].attrs['long_name'])
        fills.append(f['Data_Directional_Fields'][field].attrs['_FillValue'])
        units.append(f['Data_Directional_Fields'][field].attrs['units'])

        arrays.append(np.array(f['Data_Directional_Fields'][field]))
        
    if len(np.unique(scales)) == 1:
        scales = scales[0]
    if len(np.unique(fills)) == 1:
        fills = fills[0]
    if len(np.unique(units)) == 1:
        units = units[0]
    
    measurement_dict[cat] = {}
    measurement_dict[cat]['fields'] = fields
    measurement_dict[cat]['scale'] = scales
    measurement_dict[cat]['long_name'] = long_names
    measurement_dict[cat]['fill'] = fills
    measurement_dict[cat]['units'] = units
    measurement_dict[cat]['data'] = np.stack(arrays,axis=3)[0]

I_np
tag: I
['I1020NP', 'I443NP', 'I490P', 'I565NP', 'I670P', 'I763NP', 'I765NP', 'I865P', 'I910NP']
I1020NP, I443NP, I490P, I565NP, I670P, I763NP, I765NP, I865P, I910NP, I_p
tag: I
['I490P', 'I670P', 'I865P']
I490P, I670P, I865P, Q
tag: Q
['Q490P', 'Q670P', 'Q865P']
Q490P, Q670P, Q865P, U
tag: U
['U490P', 'U670P', 'U865P']
U490P, U670P, U865P, 

# Create Final Dictionary and Populate

In [18]:
final_dict = {}
final_dict['observation_data'] = {}
final_dict['observation_data']['I_PARASOL'] = copy.deepcopy(measurement_dict['I_np'])

## Wavelengths

In [19]:
def add_wavelengths(polarized=True):
    if polarized:
        print("polarized")
        tag = 'I_p'
        field_name = 'polarization_wavelengths'
        lambdas = [int(field.replace('I','').replace('P','')) for field in measurement_dict[tag]['fields']]

    else:
        print("non-polarized")
        tag = 'I_np'
        field_name = 'intensity_wavelengths'
        lambdas = [int(field.replace('I','').replace('NP','').replace('P','')) for field in measurement_dict[tag]['fields']]

    shape = measurement_dict[tag]['data'].shape
    new_shape = []

    lambda_arrs = []

    for lam in lambdas:    
        new_shape = []
        for dim in shape[:-1]:
            new_shape.append(dim)
        # new_shape.append(1)

        lambda_arrs.append(np.full(new_shape, fill_value = np.full(16,lam)))
        # break

    full_lambdas_arr = np.stack(lambda_arrs,axis=3)
    print(full_lambdas_arr.shape)
    
    final_dict['sensor_views_bands'] = {}
    final_dict['sensor_views_bands'][field_name] = {}
    final_dict['sensor_views_bands'][field_name]['scale'] = 1
    final_dict['sensor_views_bands'][field_name]['long_name'] = 'field_name'
    final_dict['sensor_views_bands'][field_name]['fill'] = 32767
    final_dict['sensor_views_bands'][field_name]['units'] = 'tbd'
    final_dict['sensor_views_bands'][field_name]['data'] = full_lambdas_arr
    
for i in [True,False]:
    add_wavelengths(i)

polarized
(3240, 6480, 16, 3)
non-polarized
(3240, 6480, 16, 9)


In [20]:
measurement_dict.keys()

dict_keys(['I_np', 'I_p', 'Q', 'U'])

## DOLP
* Calculate DOLP matrix
    * DOLP = sqrt(Q^2 + U^2)/I

In [21]:
scale = measurement_dict['Q']['scale']
scale
fill = measurement_dict['Q']['fill']
fill

32767

In [22]:
# scale, abs
for key in measurement_dict.keys():
    if key == 'I_p':
        I_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        I_arr[np.abs(I_arr) == fill] = 1
    
    elif key == 'Q':
        Q_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        Q_arr[np.abs(Q_arr) == fill] = 1
        
    elif key == 'U':
        U_arr = copy.deepcopy(measurement_dict[key]['data']) * scale
        U_arr[np.abs(U_arr) == fill] = 1
        
    else:
        continue

In [23]:
DOLP_arr_unfltrd = np.divide(np.sqrt(np.add(np.square(Q_arr), np.square(U_arr))), I_arr)

<ipython-input-23-a37297b360e4>:1: RuntimeWarning: divide by zero encountered in true_divide
  DOLP_arr_unfltrd = np.divide(np.sqrt(np.add(np.square(Q_arr), np.square(U_arr))), I_arr)


In [24]:
# rename - DOLP
DOLP_arr_unfltrd[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

In [25]:
indx = np.where((np.abs(I_arr) != (fill * scale)) & (np.abs(Q_arr) != (fill * scale)) & (np.abs(U_arr) != (fill * scale)))

In [26]:
# add to dictionary

final_dict['observation_data']['DOLP_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['DOLP_PARASOL']['scale'] = 1
final_dict['observation_data']['DOLP_PARASOL']['long_name'] = 'INSERT LONG NAME'
final_dict['observation_data']['DOLP_PARASOL']['fill'] = fill
final_dict['observation_data']['DOLP_PARASOL']['units'] = 'INSERT UNITS'
final_dict['observation_data']['DOLP_PARASOL']['data'] = DOLP_arr_unfltrd

## Q over I, U over I

In [27]:
Q_over_I = np.divide(Q_arr, I_arr)
U_over_I = np.divide(U_arr, I_arr)

Q_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

U_over_I[np.where((measurement_dict['I_p']['data'] == fill) | 
                          (measurement_dict['Q']['data'] == fill) | 
                          (measurement_dict['U']['data'] == fill))] = fill

final_dict['observation_data']['Q_over_I_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['Q_over_I_PARASOL']['scale'] = 1
final_dict['observation_data']['Q_over_I_PARASOL']['long_name'] = 'INSERT LONG NAME'
final_dict['observation_data']['Q_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['Q_over_I_PARASOL']['units'] = 'INSERT UNITS'
final_dict['observation_data']['Q_over_I_PARASOL']['data'] = Q_over_I

final_dict['observation_data']['U_over_I_PARASOL'] = {}
# measurement_dict[cat]['fields'] = fields
final_dict['observation_data']['U_over_I_PARASOL']['scale'] = 1
final_dict['observation_data']['U_over_I_PARASOL']['long_name'] = 'INSERT LONG NAME'
final_dict['observation_data']['U_over_I_PARASOL']['fill'] = fill
final_dict['observation_data']['U_over_I_PARASOL']['units'] = 'INSERT UNITS'
final_dict['observation_data']['U_over_I_PARASOL']['data'] = U_over_I

<ipython-input-27-6e0bec364d22>:1: RuntimeWarning: divide by zero encountered in true_divide
  Q_over_I = np.divide(Q_arr, I_arr)
<ipython-input-27-6e0bec364d22>:1: RuntimeWarning: invalid value encountered in true_divide
  Q_over_I = np.divide(Q_arr, I_arr)
<ipython-input-27-6e0bec364d22>:2: RuntimeWarning: divide by zero encountered in true_divide
  U_over_I = np.divide(U_arr, I_arr)
<ipython-input-27-6e0bec364d22>:2: RuntimeWarning: invalid value encountered in true_divide
  U_over_I = np.divide(U_arr, I_arr)


# Angles, Altitude, & Location

In [30]:
final_dict['geolocation_data'] = {}

In [31]:
f['Geolocation_Fields'].keys()

<KeysViewHDF5 ['Latitude', 'Longitude', 'column_number', 'land_sea_flag', 'row_number', 'surface_altitude']>

In [32]:
for field in ['thetas','thetav','phi']:
    if field == 'thetas':
        tag = 'solar_zenith'
    elif field == 'thetav':
        tag = 'sensor_zenith'
    elif field == 'phi':
        tag = 'relative_azimuth'
        
    final_dict['geolocation_data'][tag] = {}
    final_dict['geolocation_data'][tag]['scale'] = f['Data_Directional_Fields'][field].attrs['scale_factor']
    final_dict['geolocation_data'][tag]['long_name'] = f['Data_Directional_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['fill'] = f['Data_Directional_Fields'][field].attrs['_FillValue']
    final_dict['geolocation_data'][tag]['units'] = f['Data_Directional_Fields'][field].attrs['units']
    final_dict['geolocation_data'][tag]['data'] = np.array(f['Data_Directional_Fields'][field])

In [33]:
for field in ['Latitude','Longitude','surface_altitude']:
    if field == 'surface_altitude':
        tag = 'altitude'
    else:
        tag = field
        
    final_dict['geolocation_data'][tag] = {}
    final_dict['geolocation_data'][tag]['scale'] = f['Geolocation_Fields'][field].attrs['scale_factor']
    final_dict['geolocation_data'][tag]['long_name'] = f['Geolocation_Fields'][field].attrs['long_name']
    final_dict['geolocation_data'][tag]['fill'] = f['Geolocation_Fields'][field].attrs['_FillValue']
    final_dict['geolocation_data'][tag]['units'] = f['Geolocation_Fields'][field].attrs['units']
    final_dict['geolocation_data'][tag]['data'] = np.array(f['Geolocation_Fields'][field])

In [35]:
from netCDF4 import Dataset
import netCDF4

def write_nc(variable_dict, filename):

    try:
        nc.close()
    
    except:
        print('opening new file')
    
    nc = Dataset(filename, mode='w', format='NETCDF4')

    for cat in variable_dict.keys():

        # Create the category group to store the variables
        nc.createGroup(cat)

        for var in variable_dict[cat].keys():
            print(var)

            # Fill the dimension with variables
            dimensions = []
            for i in range(len(variable_dict[cat][var]['data'].shape)):  
                dim_name = f'{var}_{i}'
                nc.createDimension(dim_name, size=None)
                dimensions.append(dim_name)

            # Create the variable instance
            print('creating variable')
            nc[cat].createVariable(var, datatype='f4', dimensions=dimensions, fill_value=variable_dict[cat][var]['fill'])

            # Create variable metadata
            print('creating the metaverse')
            nc[cat][var].long_name = variable_dict[cat][var]['long_name']
            nc[cat][var].units = variable_dict[cat][var]['units']
            nc[cat][var].scale_factor = variable_dict[cat][var]['scale']

            # Create variable array 
            print('create variable array')
            nc[cat][var][:] = variable_dict[cat][var]['data']
            
    return nc

In [37]:
os.chdir('/gpfs/gpfs0/project/sdscap-shakeri/nasa/UVA_NASA_2021')

In [38]:
write_nc(final_dict, 'example.nc')

opening new file
I_PARASOL
creating variable
creating the metaverse
create variable array
DOLP_PARASOL
creating variable
creating the metaverse
create variable array
Q_over_I_PARASOL
creating variable
creating the metaverse
create variable array
U_over_I_PARASOL
creating variable
creating the metaverse
create variable array
intensity_wavelengths
creating variable
creating the metaverse
create variable array
solar_zenith
creating variable
creating the metaverse
create variable array
sensor_zenith
creating variable
creating the metaverse
create variable array
relative_azimuth
creating variable
creating the metaverse
create variable array
Latitude
creating variable
creating the metaverse
create variable array
Longitude
creating variable
creating the metaverse
create variable array
altitude
creating variable
creating the metaverse
create variable array


<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): I_PARASOL_0(3240), I_PARASOL_1(6480), I_PARASOL_2(16), I_PARASOL_3(9), DOLP_PARASOL_0(3240), DOLP_PARASOL_1(6480), DOLP_PARASOL_2(16), DOLP_PARASOL_3(3), Q_over_I_PARASOL_0(3240), Q_over_I_PARASOL_1(6480), Q_over_I_PARASOL_2(16), Q_over_I_PARASOL_3(3), U_over_I_PARASOL_0(3240), U_over_I_PARASOL_1(6480), U_over_I_PARASOL_2(16), U_over_I_PARASOL_3(3), intensity_wavelengths_0(3240), intensity_wavelengths_1(6480), intensity_wavelengths_2(16), intensity_wavelengths_3(9), solar_zenith_0(3240), solar_zenith_1(6480), solar_zenith_2(16), sensor_zenith_0(3240), sensor_zenith_1(6480), sensor_zenith_2(16), relative_azimuth_0(3240), relative_azimuth_1(6480), relative_azimuth_2(16), Latitude_0(3240), Latitude_1(6480), Longitude_0(3240), Longitude_1(6480), altitude_0(3240), altitude_1(6480)
    variables(dimensions): 
    groups: observation_data, sensor_views_bands, geolocation_data